# 1D_CNN model

In [1]:
import pandas as pd 
import pandas_datareader as pdr   
import numpy as np
from sklearn.preprocessing import StandardScaler

import random
import os

In [2]:
import torch
random.seed(123)
np.random.seed(123)
os.environ["PYTHONHASHSEED"] = str(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)               # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True      # type: ignore

## 데이터 로드 및 전처리

In [3]:
final = pd.read_csv('/home/jbj4278/ETH_data/final_dateadd') #feature포함 데이터

In [4]:
col = ['Date','Open', 'High', 'Low', 'Close', 'Volume','BTC_close', 'DXY', 'BTCD', 'Kimchi_premium', 'S&P500', 'Ethereum DeFi',
       'News_freq','signal']

In [5]:
final = final.loc[:,col]

In [6]:

final.loc[final['signal']==0.0 , "signal"] = 2   # 보합 > 2로
final.loc[final['signal']==1.0 , "signal"] = 0  
 # 상승 > 0으로
final.loc[final['signal']==-1.0 , "signal"] = 1 #하락 > 1로
#(상승, 하락, 보합) > (0 ,1, 2)

In [7]:
final['signal'].value_counts()


0.0    188025
1.0    171867
2.0    163209
Name: signal, dtype: int64

In [8]:
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [9]:
window_size = 10

In [10]:
len_test = int(len(final)*0.2)
test_set = final[-len_test:]
tr_set = final[:-len_test]

In [11]:
test_set.shape

(104620, 14)

In [12]:
tr_set.shape

(418481, 14)

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import wandb
!wandb login

wandb: Currently logged in as: gyeongmocho. Use `wandb login --relogin` to force relogin


In [15]:
from torchmetrics.functional import f1_score,precision_recall

In [16]:
import time

def train(dataloader):
 
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    
    loss_list = []
    for idx, (feature, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(feature.to(device))
        loss = criterion(predicted_label, label.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        
        total_count += label.size(0)
        loss_list.append(loss.item())
        
        if idx+1 % len(feature) == 0 and idx > 0:
            accuracy = total_acc/total_count
            loss_hist = np.mean(loss_list)
            elapsed = time.time() - start_time
            print(f'| epoch {epoch} | {idx}/{len(dataloader)} batches '
                f'| accuracy {accuracy} | loss {loss_hist}')
            total_acc, total_count = 0, 0
            start_time = time.time()

    wandb.log({'train_loss':np.mean(loss_list),'train_acc':total_acc/total_count})  
    return  total_acc/total_count,np.mean(loss_list)
    
            

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    loss_list = []
    
    score,precision,recall = 0, 0, 0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(dataloader):
            feature, label = feature.to(device),label.to(device)
            predicted_label = model(feature.to(device))
            loss = criterion(predicted_label, label.to(device))
            
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            loss_list.append(loss.item())
            
            
            
            score += f1_score(predicted_label.argmax(1).to(device), label,num_classes=3)
            precision += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[0]
            recall += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[1]
      
        
        score = score / len(loss_list)
        precision = precision / len(loss_list)
        recall = recall / len(loss_list)
            
    wandb.log({'val_loss':np.mean(loss_list), 'val_acc':total_acc/total_count,'f1_score':score,
              'precision':precision,'recall':recall})
    
    return total_acc/total_count, np.mean(loss_list),score


def soft_voting(dataloader):
    model.eval()
    pred_soft = []
    with torch.no_grad():
        for idx, (feature, label) in enumerate(dataloader):
            feature, label = feature.to(device),label.to(device)
            predicted_label = model(feature.to(device))
            sm = nn.Softmax(dim=1)
            probabilities = sm(predicted_label)
            prob_arr = (probabilities.detach().cpu().numpy())[0]
            pred_soft.append(prob_arr)
            
    return pred_soft

            
   

In [17]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self,path, patience=10, verbose=False, delta=0,):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [27]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)
    
        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]


In [19]:
class Conv1d(nn.Module):
    def __init__(self,outchannels, kennel_size, in_channel=12, num_class=3):
        super(Conv1d, self).__init__()
        self.outchannels = outchannels
        self.kennel_size = kennel_size
        self.conv1d_1 = nn.Conv1d(in_channels=in_channel,
                                out_channels=outchannels,
                                kernel_size=kennel_size,
                                stride=1)
        
        
                                
        self.pool = nn.AdaptiveAvgPool1d(1) #global average pooling
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

        self.dense1 = nn.Linear(outchannels,6)
        
        self.dense2 = nn.Linear(6,num_class)

    def forward(self, x):
	# Raw x shape : (B, S, F) => (B, 10, 12)
        
        # Shape : (B, F, S) => (B, 12, 10)
        x = x.transpose(1, 2)
        
        # Shape : (B, F, S) == (B, C, S) // C = channel => (B, 12, x)
        x = self.conv1d_1(x)
        
        x = self.relu(x)
        
        
        # Shape : (B, 12, 1)
        x = self.pool(x)
        
        
        
        x = self.dropout(x)
        
        
        # Shape : (B, 10)
        x = self.dense1(x.squeeze(2))
        
        x = self.relu(x)
        
        x = self.dense2(x)
        
        
        # Shape : (10, O) // O = output => (B, 3)
        
        
        

        return x

In [22]:
conv1d_1 = nn.Conv1d(in_channels=12,
                                out_channels=12,
                                kernel_size=6,
                                stride=1)

In [109]:
a = torch.randn(1,12,10)
a = conv1d_1(a)
print(a.size())

torch.Size([1, 12, 5])


In [100]:
#parameters
outchannels =  12
kennel_size = 6

lr = 0.001

In [106]:
btss = BlockingTimeSeriesSplit(n_splits=5)
n_epochs = 100
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']
score_list = []
accuracy_list = []
val_loss_list = []
voting_list = []
#cross validation

wandb.init(project='real', entity='gyeongmoCho',name='1dcnn_2fully')
#wandb.watch(model, criterion, log = "all" )


for idx,(tr_idx, val_idx) in enumerate(btss.split(tr_set)):
    
    train_data, val_data = tr_set.iloc[tr_idx], tr_set.iloc[val_idx]
    
    
    scaler = StandardScaler()
    scaler.fit(train_data[scale_cols])#for문을 돌면서 각 train, val을 스케일링ㅇ
    train_feature = scaler.transform(train_data[scale_cols])
    train_feature = pd.DataFrame(train_feature)                                 
    val_feature = scaler.transform(val_data[scale_cols])
    val_feature = pd.DataFrame(val_feature)   

    
    train_feature, train_label = make_dataset(train_feature, train_data['signal'], window_size)
    val_feature, val_label = make_dataset(val_feature, val_data['signal'], window_size)

    
    #모델 선언 
    
    model = Conv1d(outchannels, kennel_size).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0,verbose=True)
    total_accu = None
    

    print(model)
    
    early_stopping = EarlyStopping(patience = 10, verbose = True)



    #스케일링된 데이터를 torch Tensor로 변경
    val_feature = torch.FloatTensor(val_feature).to(device)
    val_label = torch.LongTensor(val_label).to(device)
    train_feature = torch.FloatTensor(train_feature).to(device)
    train_label  = torch.LongTensor(train_label).to(device)

    train_set = TensorDataset(train_feature,train_label)
    val_set = TensorDataset(val_feature,val_label)
    
    #데이터로더 사용
    BATCH_SIZE = 256
    train_dataloader = DataLoader(train_set , batch_size=BATCH_SIZE,shuffle=False)
    valid_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE,shuffle=False)

    print(f"train_feature:{train_feature.shape},val_feature:{val_feature.shape}" )
    

    for epoch in range(1, n_epochs + 1):
        # 현재 learning rate 출력
        print(optimizer.param_groups[0]['lr'])
        epoch_start_time = time.time()
        accu_train, train_loss = train(train_dataloader)
        accu_val,val_loss,score = evaluate(valid_dataloader)
        if total_accu is not None and total_accu > accu_val:
            scheduler.step()
        else:
            total_accu = accu_val
        print('-' * 59)
        print(f'| end of epoch {epoch} | time: {time.time()- epoch_start_time}s | '
            f'valid accuracy {accu_val} | valid loss {val_loss}'
            ) 

        print('-' * 59)
        
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    score_list.append(score)
    accuracy_list.append(accu_val)
    print("save score")
    voting_prob = soft_voting(valid_dataloader)
    voting_list.append(voting_prob)
    print("save voting prob")
    
    PATH = f"/home/jbj4278/ETH_data/1dcnn_2fully{idx}.pt"
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),


                }, PATH)
    print('-' * 59)
    print('Fold Finish')
    print('-' * 59)

score_list = list(map(float, score_list))
final_f1_score = np.mean(score_list)
acc_list = list(map(float, accuracy_list))
final_acc = np.mean(acc_list)
print(f"f1_score:{final_f1_score}")
print(f"accuracy:{final_acc}")
wandb.log({'mean_f1_score':final_f1_score,"mean_acc":final_acc})



        
    

   

train_acc,▁
train_loss,▁
train_acc,0.44777
train_loss,1.07711


Adjusting learning rate of group 0 to 1.0000e-03.
Conv1d(
  (conv1d_1): Conv1d(12, 12, kernel_size=(3,), stride=(1,))
  (pool): AdaptiveAvgPool1d(output_size=1)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=12, out_features=6, bias=True)
  (dense2): Linear(in_features=6, out_features=3, bias=True)
)
train_feature:torch.Size([75316, 10, 12]),val_feature:torch.Size([8360, 10, 12])
0.001
-----------------------------------------------------------
| end of epoch 1 | time: 1.2605597972869873s | valid accuracy 0.22882775119617224 | valid loss 1.2448030601848254
-----------------------------------------------------------
Validation loss decreased (inf --> 1.244803).  Saving model ...
0.001
-----------------------------------------------------------
| end of epoch 2 | time: 1.3163414001464844s | valid accuracy 0.22882775119617224 | valid loss 1.2492643594741821
-----------------------------------------------------------
EarlyStopping counter: 1 out 

-----------------------------------------------------------
| end of epoch 3 | time: 1.3853437900543213s | valid accuracy 0.30251196172248807 | valid loss 1.1114451523983118
-----------------------------------------------------------
Validation loss decreased (1.113333 --> 1.111445).  Saving model ...
0.0009997532801828658
-----------------------------------------------------------
| end of epoch 4 | time: 1.410630226135254s | valid accuracy 0.3200956937799043 | valid loss 1.1100967183257595
-----------------------------------------------------------
Validation loss decreased (1.111445 --> 1.110097).  Saving model ...
0.0009997532801828658
-----------------------------------------------------------
| end of epoch 5 | time: 1.3941001892089844s | valid accuracy 0.33026315789473687 | valid loss 1.109230399131775
-----------------------------------------------------------
Validation loss decreased (1.110097 --> 1.109230).  Saving model ...
0.0009997532801828658
----------------------------

-----------------------------------------------------------
| end of epoch 11 | time: 1.68634033203125s | valid accuracy 0.17954545454545454 | valid loss 1.1465404665831365
-----------------------------------------------------------
Validation loss decreased (1.154755 --> 1.146540).  Saving model ...
0.001
-----------------------------------------------------------
| end of epoch 12 | time: 1.3490262031555176s | valid accuracy 0.17954545454545454 | valid loss 1.1394443584210945
-----------------------------------------------------------
Validation loss decreased (1.146540 --> 1.139444).  Saving model ...
0.001
-----------------------------------------------------------
| end of epoch 13 | time: 1.3224375247955322s | valid accuracy 0.17954545454545454 | valid loss 1.1331624948617183
-----------------------------------------------------------
Validation loss decreased (1.139444 --> 1.133162).  Saving model ...
0.001
-----------------------------------------------------------
| end of epo

Adjusting learning rate of group 0 to 9.7553e-04.
-----------------------------------------------------------
| end of epoch 36 | time: 1.3774683475494385s | valid accuracy 0.35454545454545455 | valid loss 1.0887162288029988
-----------------------------------------------------------
Validation loss decreased (1.088732 --> 1.088716).  Saving model ...
0.0009755282581475767
Adjusting learning rate of group 0 to 9.7044e-04.
-----------------------------------------------------------
| end of epoch 37 | time: 1.4011404514312744s | valid accuracy 0.3541866028708134 | valid loss 1.0887655146194226
-----------------------------------------------------------
EarlyStopping counter: 1 out of 10
0.0009704403844771127
Adjusting learning rate of group 0 to 9.6489e-04.
-----------------------------------------------------------
| end of epoch 38 | time: 1.3764634132385254s | valid accuracy 0.3539473684210526 | valid loss 1.0888993360779502
-----------------------------------------------------------

## f1-score의 fold별 평균이 가장 좋았던 모델로 Test set 성능 측정
out_channels = 12,kennel_size = 6 , batch = 256, lr = 0.001

In [26]:
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']
scaler = StandardScaler()
scaler.fit(tr_set[scale_cols])#for문을 돌면서 각 train, val을 스케일링
tr_feature = scaler.transform(tr_set[scale_cols])
tr_feature = pd.DataFrame(tr_feature)                                 
test_feature = scaler.transform(test_set[scale_cols])
test_feature = pd.DataFrame(test_feature)   


test_feature, test_label = make_dataset(test_feature, test_set['signal'], window_size)


#스케일링된 데이터를 torch Tensor로 변경
test_feature = torch.FloatTensor(test_feature).to(device)
test_label  = torch.LongTensor(test_label).to(device)

final_test_set = TensorDataset(test_feature,test_label)


#데이터로더 사용
BATCH_SIZE = 256
test_dataloader = DataLoader(final_test_set , batch_size=BATCH_SIZE,shuffle=False)


In [28]:


scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'BTC_close', 'DXY', 'BTCD',
       'Kimchi_premium', 'S&P500', 'Ethereum DeFi', 'News_freq']



   
scaler = StandardScaler()
scaler.fit(tr_set[scale_cols])#for문을 돌면서 각 train, val을 스케일링ㅇ
train_feature = scaler.transform(tr_set[scale_cols])
train_feature = pd.DataFrame(train_feature)                                 
 

    
train_feature, train_label = make_dataset(train_feature, tr_set['signal'], window_size)
  

train_feature = torch.FloatTensor(train_feature).to(device)
train_label  = torch.LongTensor(train_label).to(device)

train_set = TensorDataset(train_feature,train_label)


#데이터로더 사용
BATCH_SIZE = 256
final_dataloader = DataLoader(train_set , batch_size=BATCH_SIZE,shuffle=False)




        
    

   

In [29]:
#모델 저장 경로
path = f"/home/jbj4278/ETH_data/1dcnn_final_model.pt"

In [31]:
#best parameter
#parameters
outchannels =  12
kennel_size = 6

lr = 0.001

In [32]:
model = Conv1d(outchannels, kennel_size).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0,verbose=True)


Adjusting learning rate of group 0 to 1.0000e-03.


In [33]:
import time

def train(dataloader):
 
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    
    loss_list = []
    for idx, (feature, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(feature.to(device))
        loss = criterion(predicted_label, label.to(device))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        
        total_count += label.size(0)
        loss_list.append(loss.item())
        
        if idx+1 % len(feature) == 0 and idx > 0:
            accuracy = total_acc/total_count
            loss_hist = np.mean(loss_list)
            elapsed = time.time() - start_time
            print(f'| epoch {epoch} | {idx}/{len(dataloader)} batches '
                f'| accuracy {accuracy} | loss {loss_hist}')
            total_acc, total_count = 0, 0
            start_time = time.time()

      
    return  total_acc/total_count,np.mean(loss_list)
    
            

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    loss_list = []
    pred_label = []
    pred_prob = []
    score,precision,recall = 0, 0, 0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(dataloader):
            feature, label = feature.to(device),label.to(device)
            predicted_label = model(feature.to(device))
            loss = criterion(predicted_label, label.to(device))
            
            
            sm = nn.Softmax(dim=1)
            probabilities = sm(predicted_label)
            prob_arr = (probabilities.detach().cpu().numpy())[0]
            pred_prob.append(prob_arr)
            
            
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            loss_list.append(loss.item())
            pred_label.append(predicted_label.argmax(1))
            
            
            score += f1_score(predicted_label.argmax(1).to(device), label,num_classes=3)
            precision += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[0]
            recall += precision_recall(predicted_label.argmax(1).to(device), label, average='macro', num_classes=3)[1]
      
        
        score = score / len(loss_list)
        precision = precision / len(loss_list)
        recall = recall / len(loss_list)
            
    
    
    return total_acc/total_count, np.mean(loss_list),score,pred_label,pred_prob


            
   

In [36]:
n_epochs = 100
early_stopping = EarlyStopping(path,patience = 3, verbose = True)


for epoch in range(1, n_epochs + 1):
    # 현재 learning rate 출력
    print(optimizer.param_groups[0]['lr'])
    epoch_start_time = time.time()
    accu_train, train_loss = train(final_dataloader)
    accu_test,test_loss,score,label,prob = evaluate(test_dataloader)
    if total_accu is not None and total_accu > accu_test:
        scheduler.step()
    else:
        total_accu = accu_test
    print('-' * 59)
    print(f'| end of epoch {epoch} | time: {time.time()- epoch_start_time}s | '
        f'valid accuracy {accu_test} | valid loss {test_loss}'
        ) 

    print('-' * 59)

    early_stopping(test_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break
print(score)
torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),


                }, path)
    

0.001
-----------------------------------------------------------
| end of epoch 1 | time: 8.334758043289185s | valid accuracy 0.32187171398527864 | valid loss 1.1021751418964787
-----------------------------------------------------------
Validation loss decreased (inf --> 1.102175).  Saving model ...
0.001
-----------------------------------------------------------
| end of epoch 2 | time: 8.452173233032227s | valid accuracy 0.32993977631201604 | valid loss 1.099953013847976
-----------------------------------------------------------
Validation loss decreased (1.102175 --> 1.099953).  Saving model ...
0.001
-----------------------------------------------------------
| end of epoch 3 | time: 8.351358890533447s | valid accuracy 0.3355128572794188 | valid loss 1.0988157411950725
-----------------------------------------------------------
Validation loss decreased (1.099953 --> 1.098816).  Saving model ...
0.001
Adjusting learning rate of group 0 to 9.9975e-04.
---------------------------

In [37]:
wandb.init(project='result', entity='gyeongmoCho',name='1DCNN')
wandb.log({'final_f1_score':score})

wandb: Currently logged in as: gyeongmocho. Use `wandb login --relogin` to force relogin


In [38]:
pred_label = list(map(list, label))

answer = sum(pred_label, [])

final_label = []
for i in answer:
    final_label.append(i.tolist())

In [39]:
def make_result(model,test_dataloader):
    model.eval()
    pred_soft = []
    pred_hard=[]
    
    correct_count, all_count = 0,0
    with torch.no_grad():
        for idx, (feature, label) in enumerate(test_dataloader):
            if torch.cuda.is_available():
                feature, labels = feature.to(device),label.to(device)
            for i in range(len(labels)):
                #soft
                predicted_label = model(feature.to(device))
                sm = nn.Softmax(dim=1)
                probabilities = sm(predicted_label)
                prob_arr = (probabilities.detach().cpu().numpy())[0]
                pred_soft.append(prob_arr)
                #hard
                logps = model(feature.to(device))
                ps = torch.exp(logps)
                probab = list(ps.cpu()[0])
                pred_label = probab.index(max(probab))
                true_label = labels.cpu()[i]
                pred_hard.append(pred_label)
                if(true_label == pred_label):
                    correct_count += 1
                all_count += 1
                
                
    return pred_soft ,pred_hard




In [40]:
soft, hard = make_result(model,test_dataloader)

In [41]:
soft = pd.DataFrame(soft)

In [42]:
final_labels = pd.DataFrame(final_label,columns = ['pred_labels'])

test_set = test_set[:-10]

index = final_labels.index

test_set.index = index

plus_label_df = test_set.join(final_labels['pred_labels'],how='right') 

plus_label_df = plus_label_df.join(soft,how='right') 




In [43]:
plus_label_df.to_csv('final_1DCNN_label.csv')

In [45]:
plus_label_df['pred_labels'].value_counts()


2    84626
1    11720
0     8264
Name: pred_labels, dtype: int64